In [1]:
# import dependencies
import pandas as pd 
import gmaps 
import requests 
#import api key
from config1 import g_key

In [2]:
# store cities csv into a dataframe
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rainfall,Snowfall
0,0,Rikitea,PF,2020-07-13 01:46:32,-23.12,-134.97,70.93,75,54,13.18,broken clouds,0.0,0
1,1,Charters Towers,AU,2020-07-13 01:46:32,-20.10,146.27,66.83,30,39,16.58,scattered clouds,0.0,0
2,2,Vaini,TO,2020-07-13 01:46:33,-21.20,-175.20,77.00,78,75,17.22,light intensity shower rain,0.0,0
3,3,Cape Town,ZA,2020-07-13 01:54:16,-33.93,18.42,59.00,67,75,31.09,broken clouds,0.0,0
4,4,Mataura,NZ,2020-07-13 01:54:32,-46.19,168.86,45.00,83,83,4.92,broken clouds,0.0,0


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Date                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Weather Description     object
Rainfall               float64
Snowfall                 int64
dtype: object

In [13]:
# Prompt the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rainfall = input("Do you want it to be raining? (yes/no) ")
snowfall = input("Do you want it to be snowing? (yes/no) ")

In [16]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                    (city_data_df["Max Temp"] >= min_temp) & 
                    (city_data_df["Rainfall"]==0 
                    if rainfall == "no"
                    else city_data_df["Rainfall"] > 0) & 
                    (city_data_df["Snowfall"]==0 
                    if snowfall == "no" 
                    else city_data_df["Snowfall"] > 0)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rainfall,Snowfall
2,2,Vaini,TO,2020-07-13 01:46:33,-21.20,-175.20,77.00,78,75,17.22,light intensity shower rain,0.0,0
7,7,Nizhniy Kuranakh,RU,2020-07-13 01:49:44,58.84,125.49,88.61,40,63,3.89,broken clouds,0.0,0
10,10,Cockburn Town,TC,2020-07-13 01:46:36,21.46,-71.14,83.23,79,0,15.73,clear sky,0.0,0
13,13,Mayo,US,2020-07-13 01:54:34,38.89,-76.51,82.40,65,20,10.29,few clouds,0.0,0
17,17,Tiarei,PF,2020-07-13 01:49:54,-17.53,-149.33,80.60,69,75,16.11,broken clouds,0.0,0


In [17]:
len(preferred_cities_df)

177

In [18]:
# Configure gmaps to use google api key
gmaps.configure(api_key=g_key)

In [21]:
# Create DF called hotel_df to store hotel names with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Weather Description", "Rainfall", "Snowfall"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Weather Description,Rainfall,Snowfall,Hotel Name
2,Vaini,TO,77.00,-21.20,-175.20,light intensity shower rain,0.0,0,
7,Nizhniy Kuranakh,RU,88.61,58.84,125.49,broken clouds,0.0,0,
10,Cockburn Town,TC,83.23,21.46,-71.14,clear sky,0.0,0,
13,Mayo,US,82.40,38.89,-76.51,few clouds,0.0,0,
17,Tiarei,PF,80.60,-17.53,-149.33,broken clouds,0.0,0,
18,Umm Lajj,SA,81.88,25.02,37.27,broken clouds,0.0,0,
27,Tateyama,JP,75.99,34.98,139.87,broken clouds,0.0,0,
31,Hithadhoo,MV,84.43,-0.60,73.08,scattered clouds,0.0,0,
32,Thinadhoo,MV,84.99,0.53,72.93,broken clouds,0.0,0,
35,Oistins,BB,82.40,13.07,-59.53,scattered clouds,0.0,0,


In [22]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [23]:
# Iterate through the Dataframe
for index, row in hotel_df.iterrows():
    # Get lat and lng
    lat = row["Lat"]
    lng = row["Lng"]

    # Add lat and lng to location key for params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Use search term: "lodging" and our lat and lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Weather Description,Rainfall,Snowfall,Hotel Name
2,Vaini,TO,77.00,-21.20,-175.20,light intensity shower rain,0.0,0,Keleti Beach Resort
7,Nizhniy Kuranakh,RU,88.61,58.84,125.49,broken clouds,0.0,0,Park Pobedy
10,Cockburn Town,TC,83.23,21.46,-71.14,clear sky,0.0,0,Osprey Beach Hotel
13,Mayo,US,82.40,38.89,-76.51,few clouds,0.0,0,Resorts Resource Group International
17,Tiarei,PF,80.60,-17.53,-149.33,broken clouds,0.0,0,Le Rocher de Tahiti
18,Umm Lajj,SA,81.88,25.02,37.27,broken clouds,0.0,0,Moon light Furnished Units
27,Tateyama,JP,75.99,34.98,139.87,broken clouds,0.0,0,Tateyama Seaside Hotel
31,Hithadhoo,MV,84.43,-0.60,73.08,scattered clouds,0.0,0,Scoop Guest House
32,Thinadhoo,MV,84.99,0.53,72.93,broken clouds,0.0,0,
35,Oistins,BB,82.40,13.07,-59.53,scattered clouds,0.0,0,Butterfly Beach Hotel


In [43]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
info_box_template = """
<dl>
<dt>Hotel Name:</dt><dd>{Hotel Name}</dd>
<dt>City:</dt><dd>{City}</dd>
<dt>Country:</dt><dd>{Country}</dd>
<dt>Weather Description:</dt/<dd>{Weather Description}</dd>
<dt>Max Temp:</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [37]:
# Store dataframe row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [40]:
# Create the output file (CSV).
output_data_file1 = "weather_data/WeatherPy_vacation.csv"
# Export the filtered data into a CSV.
hotel_df.to_csv(output_data_file1, index_label="City_ID")